# Experiments
In this notebook we will cover some advanced configurations for Experiment. 
1. **Experiment Creation**: Creating an Experiment from scratch.
2. **Experiment Running**: Running an experiment and evaluates its learners on its environments.
5. **Experiment Multiprocessing**: Controlling execution to manage resources and optimize runtime.
6. **Experiment Restoration**: Saving results to file as an Experiment evaluates so that work isn't loss to unexpected failures.

## Experiment Creation
Before you do anything with an **Experiment** you have to create one. To create an **Experiment** we need a list of Environments and Learners:

In [1]:
import coba as cb

#this creates our experiment. There are more options that can be passed here. These are discussed further down.
environments = cb.Environments.from_linear_synthetic(500).binary()
learners     = [cb.VowpalEpsilonLearner(),cb.RandomLearner()]
experiment   = cb.Experiment(environments,learners)

## Experiment Evaluation
Once an **Experiment** has been created the next step is to use it to evaluate it. This looks like this:

2022-11-18 10:10:09 -- Processing chunk...
2022-11-18 10:10:09 --   * Recording Learner 0 parameters... (0.0 seconds) (completed)
2022-11-18 10:10:09 --   * Recording Learner 1 parameters... (0.0 seconds) (completed)
2022-11-18 10:10:09 --   * Loading LinearSynth(A=5,c=5,a=5,R=['a', 'xa'],seed=1)... (0.1 seconds) (completed)
2022-11-18 10:10:09 --   * Creating Environment 0 from Loaded Source... (0.0 seconds) (completed)
2022-11-18 10:10:09 --   * Recording Environment 0 statistics... (0.0 seconds) (completed)
2022-11-18 10:10:09 --   * Evaluating Learner 0 on Environment 0... (0.16 seconds) (completed)
2022-11-18 10:10:09 --   * Evaluating Learner 1 on Environment 0... (0.0 seconds) (completed)


In [3]:
import coba as cb

#this creates our experiment. There are more options that can be passed here. These are discussed further down.
environments = cb.Environments.from_linear_synthetic(500).binary()
learners     = [cb.VowpalEpsilonLearner(),cb.RandomLearner()]
experiment   = cb.Experiment(environments,learners)

experiment.run().plot_learners(xlim=(10,500))

## Experiment Multiprocessing

The **Experiment** class comes with highly configurable multiprocessing functionality. There are three parameters which control how **Experiment** orchestrates tasks across process:
1. `processes` Determines how many processes to use when executing an experiment.
2. `maxtasksperchild` Determines how many "tasks" processes should complete before being replaced with new processes
3. `chunk_by` Determines how tasks (i.e., environment/learner pairs) are chunked when passed to processeses for evaluation

These parameters can be set in one of three way: 
1. A coba configuration file containing the following (more information about that in the configurations notebook)
```json
{ "experiment": {"processes":1, "maxtasksperchild":0, "chunk_by":"source" } }
```
2. Programatically using the **CobaContext** interface
```python
 from coba.contexts import CobaContext

 CobaContext.experiment.processes=1
 CobaContext.experiment.maxtasksperchild= 0
 CobaContext.experiment.chunk_by = 'source'
```
3. Directly on an **Experiment**
```python
 Experiment(environments,learners).config(processes=1,chunk_by='source',maxtasksperchild=1).evaluate()
```

By default **Experiment** multiprocessing settings are `processes=1, maxtasksperchild=0, chunk_by='source'`, which means only one process will be used. Multiple processes will only be used if `processes>1` or `maxtasksperchild>0`. The `chunk_by` parameter is only relevant when multiple processes are used. Otherwise it is ignored. Below are a few examples.

Wall time: 3.38 s


In [13]:
import coba as cb

environments = cb.Environments.from_linear_synthetic(2000).shuffle([1,2,3,4,5])
learners    = [cb.VowpalEpsilonLearner(),cb.RandomLearner()]

#single processing, chunk_by is ignored (i.e., everything is in one chunk)
cb.Experiment(environments, learners).run(quiet=True).plot_learners(err='se')

And below is an example of a Experiment that processes source chunks in parallel. Notice that when multiple processesors are executing the logging output now contains which processor sent the message in addition to when the message was sent. Also notice the difference in execution time between the single process and multiprocess runs. This example still only has one evaluation chunk because the three shuffle environments all come from the same "source".

Wall time: 3.71 s


In [14]:
import coba as cb

environments = cb.Environments.from_linear_synthetic(2000).shuffle([1,2,3,4,5])
learners    = [cb.VowpalEpsilonLearner(),cb.RandomLearner()]

cb.Experiment(environments, learners).config(processes=2, chunk_by='source').run(quiet=True).plot_learners(err='se')

Finally, if one wants to evaluate an Experiment in full parallel `chunk_by` can be set to `'task'`. This means each chunk will be a single task. This can drastically speed up evaluations when Learners are the processing bottleneck. Unfortunately this also means we'll end up doing some duplicate work. Notice that our environment has to be reloaded every time a task is processed.

In [17]:
import coba as cb

environments = cb.Environments.from_linear_synthetic(2000).shuffle([1,2,3,4,5])
learners    = [cb.VowpalEpsilonLearner(),cb.RandomLearner()]

cb.Experiment(environments, learners).config(processes=4, chunk_by='task').run(quiet=True).plot_learners(err='se')

## Experiment Restoration

Experiments come with one final bit of functionality: if provided a file path Experiments will write their results to file as they run. This can be handy in the case of something interrupting an **Experiment** during evaluation (e.g., the internet going down or a computer being forced to restart) because the **Experiment** will be able to resume at the point of interuption. The **Experiment** restoration file is also useful for later analysis since a **Result** object can be created from it. Here's what it looks like to create a restoration/result file. Notice that if you run this code block twice the second time there will be no output due to restoration. (If you are runing this code on mybinder this may or may not work as file write permissions can be blocked)

2022-10-29 18:33:42 -- Restoring existing experiment logs...


In [37]:
import coba as cb

environments = cb.Environments.from_linear_synthetic(2000).shuffle([1,2,3,4,5])
learners    = [cb.VowpalEpsilonLearner(),cb.RandomLearner()]

cb.Experiment(environments, learners).run('example.log').plot_learners()